In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.append('/nfs/home/users/krakovic/sharedscratch/notebooks/latticea_he/libraries/')
from clinical import load_clinical
from data_processing import *

In [3]:
## Import the leiden clusters from the H&Es

clusters_frame_bioclavis = pd.read_csv('/mnt/cephfs/home/users/krakovic/sharedscratch/Histomorphological-Phenotype-Learning/results/BarlowTwins_3/LATTICeA_5x/h224_w224_n3_zdim128_filtered/rapids_2p5m/adatas/LATTICeA_BioclavisHE_4pc_5x_he_train_filtered_leiden_2p5__fold2.csv')


clusters_frame_bioclavis['core_ID'] = clusters_frame_bioclavis['slides'].apply(lambda x: x.split('_')[2])
clusters_frame_bioclavis['TMA'] = clusters_frame_bioclavis['core_ID'].apply(lambda x: x.split('-')[0])
clusters_frame = clusters_frame_bioclavis.copy()
leiden_clusters = np.unique(clusters_frame['leiden_2.5'])
clusters_frame['core'] = clusters_frame['slides'].apply(lambda x: x[9:])

clusters_frame = clusters_frame.copy(deep=True)
clusters_frame

,Unnamed: 0,hist_subtype,labels,patterns,samples,slides,tiles,leiden_2.5,os_event_ind,os_event_data,core_ID,TMA,core
0,0,NaN,0.0,NaN,ACA_0224,ACA_0224_04-07-H,0_1.jpeg,21,0,62.005479,04-07-H,04,04-07-H
1,1,NaN,0.0,NaN,ACA_0224,ACA_0224_04-07-H,1_0.jpeg,50,0,62.005479,04-07-H,04,04-07-H
2,2,NaN,0.0,NaN,ACA_0224,ACA_0224_04-07-H,0_0.jpeg,64,0,62.005479,04-07-H,04,04-07-H
3,3,NaN,0.0,NaN,ACA_0224,ACA_0224_04-07-H,1_1.jpeg,21,0,62.005479,04-07-H,04,04-07-H
4,4,NaN,0.0,NaN,ACA_0224,ACA_0224_04-06-H,0_1.jpeg,54,0,62.005479,04-06-H,04,04-06-H
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8370,8370,NaN,0.0,NaN,ACA_0347,ACA_0347_04-16-C,1_1.jpeg,12,1,2.465753,04-16-C,04,04-16-C
8371,8371,NaN,0.0,NaN,ACA_0896,ACA_0896_18-04-I,0_1.jpeg,33,1,0.164384,18-04-I,18,18-04-I
8372,8372,NaN,0.0,NaN,ACA_0896,ACA_0896_18-04-I,1_0.jpeg,33,1,0.164384,18-04-I,18,18-04-I
8373,8373,NaN,0.0,NaN,ACA_0896,ACA_0896_18-04-I,0_0.jpeg,33,1,0.164384,18-04-I,18,18-04-I


In [4]:
pivot_clusters_frame = pd.get_dummies(clusters_frame[['samples', 'core_ID', 'leiden_2.5']], prefix='cluster', columns=['leiden_2.5'])
pivot_clusters_frame = pivot_clusters_frame.groupby(by='core_ID').sum().reset_index()

leiden_clusters, malignant_clusters, stromal_clusters = load_topography()
malignant_clusters = list(malignant_clusters)+[10,16]

majority_clusters_frame = pivot_clusters_frame.copy(deep=True)

majority_clusters_frame = majority_clusters_frame[['core_ID']+[f'cluster_{col}' for col in malignant_clusters]]
majority_clusters_frame['max'] = majority_clusters_frame[[f'cluster_{col}' for col in malignant_clusters]].max(numeric_only=True, axis=1)
majority_clusters_frame['sum'] = majority_clusters_frame[[f'cluster_{col}' for col in malignant_clusters]].sum(numeric_only=True, axis=1)

cluster_pure_cores = majority_clusters_frame.loc[(majority_clusters_frame['max'] / majority_clusters_frame['sum']) > 0.5]
cluster_pure_cores['cluster'] = cluster_pure_cores[[f'cluster_{col}' for col in malignant_clusters]].idxmax(numeric_only=True, axis=1)
cluster_pure_cores['cluster'] = cluster_pure_cores['cluster'].apply(lambda x: 'HPC ' + str(x.split("_")[1]))
cluster_pure_cores['cluster'].value_counts()

/tmp/ipykernel_427914/4112667579.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pivot_clusters_frame = pivot_clusters_frame.groupby(by='core_ID').sum().reset_index()
/tmp/ipykernel_427914/4112667579.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_pure_cores['cluster'] = cluster_pure_cores[[f'cluster_{col}' for col in malignant_clusters]].idxmax(numeric_only=True, axis=1)
/tmp/ipykernel_427914/4112667579.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

HPC 52    115
HPC 50     69
HPC 11     63
HPC 64     56
HPC 46     54
HPC 35     54
HPC 44     47
HPC 16     44
HPC 67     42
HPC 41     41
HPC 68     40
HPC 48     39
HPC 0      32
HPC 27     31
HPC 17     28
HPC 69     27
HPC 31     24
HPC 34     23
HPC 28     23
HPC 10     21
HPC 40     21
HPC 30     20
HPC 21     20
HPC 13     18
HPC 61     18
HPC 33     15
HPC 47     14
HPC 54     13
HPC 5      12
Name: cluster, dtype: int64

In [5]:
cluster_pure_cores

,core_ID,cluster_0,cluster_5,cluster_11,cluster_13,cluster_17,cluster_21,cluster_27,cluster_28,cluster_30,...,cluster_61,cluster_64,cluster_67,cluster_68,cluster_69,cluster_10,cluster_16,max,sum,cluster
1,01-02-B,0,0,0,0,0,0,0,0,0,...,0,3,0,1,0,0,0,3,4,HPC 64
6,01-02-G,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,HPC 69
9,01-03-C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,HPC 50
10,01-03-E,0,0,0,0,0,0,2,0,1,...,0,0,0,0,0,0,0,2,3,HPC 27
14,01-04-A,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,3,4,HPC 21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2224,23-13-H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,4,HPC 33
2226,23-14-A,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,3,3,HPC 30
2227,23-14-B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,4,HPC 52
2232,23-14-J,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,4,HPC 11


In [6]:
def concat_phenotypes(row, colnames):
    positive_cols = [col for col in colnames if row[col] == 1]
    if len(positive_cols) == 0:
        return 'Negative'
    return '_'.join(['HPC ' + col.split("_")[1] for col in positive_cols])

# May take 'mixed' cases if the epithelial compartment is pure is. if the other HPC is a stromal one
colnames = [c for c in majority_clusters_frame.columns.values if c.startswith('cluster_')]
borderline_cores = majority_clusters_frame.loc[(majority_clusters_frame['max'] == 2)]
borderline_cores['hpc_phenotype'] = borderline_cores.apply(lambda row: concat_phenotypes(row, colnames), axis=1)
borderline_cores

/tmp/ipykernel_427914/893914929.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  borderline_cores['hpc_phenotype'] = borderline_cores.apply(lambda row: concat_phenotypes(row, colnames), axis=1)


,core_ID,cluster_0,cluster_5,cluster_11,cluster_13,cluster_17,cluster_21,cluster_27,cluster_28,cluster_30,...,cluster_61,cluster_64,cluster_67,cluster_68,cluster_69,cluster_10,cluster_16,max,sum,hpc_phenotype
0,01-02-A,0,0,0,0,0,0,0,0,0,...,1,0,0,0,2,0,0,2,4,HPC 48_HPC 61
2,01-02-C,0,0,0,0,0,0,1,0,0,...,0,2,0,0,0,0,0,2,4,HPC 27_HPC 31
8,01-03-B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2,4,HPC 46_HPC 69
10,01-03-E,0,0,0,0,0,0,2,0,1,...,0,0,0,0,0,0,0,2,3,HPC 30
15,01-04-B,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,2,3,HPC 46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2213,23-10-J,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,2,4,Negative
2215,23-11-D,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,4,HPC 31_HPC 52
2218,23-11-G,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,2,Negative
2231,23-14-I,2,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,2,4,Negative


In [7]:
def check_companion_phenotype(row, malignant_clusters, stromal_clusters):
    clus1 = int(row['hpc_phenotype'].split("_")[0].split(" ")[1])
    clus2 = int(row['hpc_phenotype'].split("_")[1].split(" ")[1])


    if clus1 in malignant_clusters and clus2 in malignant_clusters:
        return -1
    
    elif clus1 in stromal_clusters and clus2 in malignant_clusters:
        return clus2
    
    elif clus1 in malignant_clusters and clus2 in stromal_clusters:
        return clus1
    
    else:
        return -1
    
borderline_cores = borderline_cores[borderline_cores['hpc_phenotype'] != 'Negative'] # filter those without hits
borderline_cores['single_phenotype'] = borderline_cores['hpc_phenotype'].apply(lambda x: 0 if '_' in x else 1)
borderline_cores = borderline_cores[borderline_cores['single_phenotype'] == 0]
borderline_cores['malignant_phenotype'] = borderline_cores.apply(lambda row: check_companion_phenotype(row, malignant_clusters, stromal_clusters), axis=1)
borderline_cores

/tmp/ipykernel_427914/2224595421.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  borderline_cores['single_phenotype'] = borderline_cores['hpc_phenotype'].apply(lambda x: 0 if '_' in x else 1)


,core_ID,cluster_0,cluster_5,cluster_11,cluster_13,cluster_17,cluster_21,cluster_27,cluster_28,cluster_30,...,cluster_67,cluster_68,cluster_69,cluster_10,cluster_16,max,sum,hpc_phenotype,single_phenotype,malignant_phenotype
0,01-02-A,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,2,4,HPC 48_HPC 61,0,-1
2,01-02-C,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,2,4,HPC 27_HPC 31,0,-1
8,01-03-B,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,2,4,HPC 46_HPC 69,0,-1
18,01-04-E,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,2,4,HPC 31_HPC 68,0,-1
21,01-05-A,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,2,4,HPC 52_HPC 69,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177,23-06-A,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,2,4,HPC 28_HPC 34,0,-1
2181,23-06-F,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,2,4,HPC 64_HPC 69,0,-1
2195,23-09-B,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,2,4,HPC 41_HPC 54,0,-1
2205,23-10-B,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,2,4,HPC 11_HPC 44,0,-1


In [8]:
# Revised classification -- see notes for details on rationale
def assign_supercluster(x):
    if x in ['HPC 67', 'HPC 0', 'HPC 50', 'HPC 44']: # n=4
        return 'Hot, cohesive'
    elif x in ['HPC 10', 'HPC 16', 'HPC 28', 'HPC 61', 'HPC 69', 'HPC 48', 'HPC 13']: # n=7
        return 'Hot, discohesive'
    elif x in ['HPC 33', 'HPC 21', 'HPC 47', 'HPC 17', 'HPC 11', 'HPC 68', 'HPC 34']: # n=7
        return 'Cold, cohesive'
    elif x in ['HPC 30', 'HPC 40', 'HPC 27', 'HPC 64', 'HPC 5', 'HPC 31']: # n=6
        return 'Cold, discohesive'
    else:
        return 'Unknown'
    
# def assign_supercluster(x): previous iteration
#     if x in ['HPC 69', 'HPC 67', 'HPC 0', 'HPC 50', 'HPC 44']:
#         return 'Hot, cohesive'
#     elif x in ['HPC 10', 'HPC 16', 'HPC 28', 'HPC 61', 'HPC 48', 'HPC 13', 'HPC 5', 'HPC 31']:
#         return 'Hot, discohesive'
#     elif x in ['HPC 33', 'HPC 21', 'HPC 47', 'HPC 17', 'HPC 52', 'HPC 11', 'HPC 68', 'HPC 34']:
#     # elif x in ['HPC 33', 'HPC 21', 'HPC 47', 'HPC 17', 'HPC 52', 'HPC 41', 'HPC 46', 'HPC 11', 'HPC 54', 'HPC 68', 'HPC 34']:
#         return 'Cold, cohesive'
#     elif x in ['HPC 30', 'HPC 40', 'HPC 27', 'HPC 64']:
#         return 'Cold, discohesive'
#     else:
#         return 'Unknown'
    
def count_hot_cohesive(row): # n=4
    return row['cluster_67'] + row['cluster_0'] + row['cluster_50'] + row['cluster_44']

def count_hot_discohesive(row): # n=7
    return row['cluster_10'] + row['cluster_16'] + row['cluster_28'] + row['cluster_61'] + row['cluster_48'] + row['cluster_13'] + row['cluster_69']
    # return row['cluster_10'] + row['cluster_16'] + row['cluster_28'] + row['cluster_61'] + row['cluster_48'] + row['cluster_13'] + row['cluster_5'] + row['cluster_35'] + row['cluster_31']

def count_cold_cohesive(row): # n=7
    return row['cluster_33'] + row['cluster_21'] + row['cluster_47'] + row['cluster_17'] + row['cluster_11'] + row['cluster_68'] + row['cluster_34']

def count_cold_discohesive(row): # n=6
    return row['cluster_30'] + row['cluster_40'] + row['cluster_27'] + row['cluster_64'] + row['cluster_5'] + row['cluster_31']

majority_clusters_frame['hot_cohesive'] = majority_clusters_frame.apply(lambda row: count_hot_cohesive(row), axis=1)
majority_clusters_frame['hot_discohesive'] = majority_clusters_frame.apply(lambda row: count_hot_discohesive(row), axis=1)
majority_clusters_frame['cold_cohesive'] = majority_clusters_frame.apply(lambda row: count_cold_cohesive(row), axis=1)
majority_clusters_frame['cold_discohesive'] = majority_clusters_frame.apply(lambda row: count_cold_discohesive(row), axis=1)


In [9]:
majority_clusters_frame

,core_ID,cluster_0,cluster_5,cluster_11,cluster_13,cluster_17,cluster_21,cluster_27,cluster_28,cluster_30,...,cluster_68,cluster_69,cluster_10,cluster_16,max,sum,hot_cohesive,hot_discohesive,cold_cohesive,cold_discohesive
0,01-02-A,0,0,0,0,0,0,0,0,0,...,0,2,0,0,2,4,0,4,0,0
1,01-02-B,0,0,0,0,0,0,0,0,0,...,1,0,0,0,3,4,0,0,1,3
2,01-02-C,0,0,0,0,0,0,1,0,0,...,0,0,0,0,2,4,0,0,0,4
3,01-02-D,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,4,0,0,1,3
4,01-02-E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,23-15-D,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,4,0,0,2,1
2236,23-15-G,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2237,23-15-H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,4,2,1,1,0
2238,23-15-I,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
majority_clusters_frame['sc_max'] = majority_clusters_frame[['hot_cohesive', 'hot_discohesive', 'cold_cohesive', 'cold_discohesive']].max(numeric_only=True, axis=1)
majority_clusters_frame

,core_ID,cluster_0,cluster_5,cluster_11,cluster_13,cluster_17,cluster_21,cluster_27,cluster_28,cluster_30,...,cluster_69,cluster_10,cluster_16,max,sum,hot_cohesive,hot_discohesive,cold_cohesive,cold_discohesive,sc_max
0,01-02-A,0,0,0,0,0,0,0,0,0,...,2,0,0,2,4,0,4,0,0,4
1,01-02-B,0,0,0,0,0,0,0,0,0,...,0,0,0,3,4,0,0,1,3,3
2,01-02-C,0,0,0,0,0,0,1,0,0,...,0,0,0,2,4,0,0,0,4,4
3,01-02-D,0,0,0,0,0,0,1,0,0,...,0,0,0,1,4,0,0,1,3,3
4,01-02-E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,23-15-D,0,0,0,0,0,0,0,0,0,...,0,0,0,1,4,0,0,2,1,2
2236,23-15-G,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2237,23-15-H,0,0,0,0,0,0,0,0,0,...,0,0,0,1,4,2,1,1,0,2
2238,23-15-I,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
supercluster_pure_cores = majority_clusters_frame.loc[(majority_clusters_frame['sc_max'] / majority_clusters_frame['sum']) > 0.5]
supercluster_pure_cores['supercluster'] = supercluster_pure_cores[['hot_cohesive', 'hot_discohesive', 'cold_cohesive', 'cold_discohesive']].idxmax(numeric_only=True, axis=1)
# supercluster_pure_cores['supercluster'] = supercluster_pure_cores['supercluster'].apply(lambda x: x.split("_")[0].capitalize())
supercluster_pure_cores['supercluster'].value_counts()

/tmp/ipykernel_427914/785160789.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  supercluster_pure_cores['supercluster'] = supercluster_pure_cores[['hot_cohesive', 'hot_discohesive', 'cold_cohesive', 'cold_discohesive']].idxmax(numeric_only=True, axis=1)


hot_discohesive     283
cold_discohesive    269
cold_cohesive       261
hot_cohesive        245
Name: supercluster, dtype: int64

In [12]:
supercluster_pure_cores = supercluster_pure_cores.merge(cluster_pure_cores[['core_ID', 'cluster']], on='core_ID', how='left')

In [13]:
supercluster_pure_cores

,core_ID,cluster_0,cluster_5,cluster_11,cluster_13,cluster_17,cluster_21,cluster_27,cluster_28,cluster_30,...,cluster_16,max,sum,hot_cohesive,hot_discohesive,cold_cohesive,cold_discohesive,sc_max,supercluster,cluster
0,01-02-A,0,0,0,0,0,0,0,0,0,...,0,2,4,0,4,0,0,4,hot_discohesive,NaN
1,01-02-B,0,0,0,0,0,0,0,0,0,...,0,3,4,0,0,1,3,3,cold_discohesive,HPC 64
2,01-02-C,0,0,0,0,0,0,1,0,0,...,0,2,4,0,0,0,4,4,cold_discohesive,NaN
3,01-02-D,0,0,0,0,0,0,1,0,0,...,0,1,4,0,0,1,3,3,cold_discohesive,NaN
4,01-02-F,0,1,1,0,0,0,0,0,0,...,0,1,3,0,0,1,2,2,cold_discohesive,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,23-13-H,0,0,0,0,0,0,0,0,0,...,0,3,4,1,0,3,0,3,cold_cohesive,HPC 33
1054,23-14-A,0,0,0,0,0,0,0,0,3,...,0,3,3,0,0,0,3,3,cold_discohesive,HPC 30
1055,23-14-D,0,0,0,0,0,0,0,0,0,...,0,1,2,0,0,0,2,2,cold_discohesive,NaN
1056,23-14-J,0,0,3,0,0,0,0,0,0,...,0,3,4,0,0,3,0,3,cold_cohesive,HPC 11


In [16]:
supercluster_pure_cores[['core_ID', 'supercluster', 'cluster']].to_csv('/nfs/home/users/krakovic/sharedscratch/notebooks/latticea_he/final_figures/bioclavis/data/bioclavis_pure_superclusters.csv')

In [14]:
supercluster_pure_cores[['core_ID', 'supercluster', 'cluster']]

,core_ID,supercluster,cluster
0,01-02-A,hot_discohesive,NaN
1,01-02-B,cold_discohesive,HPC 64
2,01-02-C,cold_discohesive,NaN
3,01-02-D,cold_discohesive,NaN
4,01-02-F,cold_discohesive,NaN
...,...,...,...
1053,23-13-H,cold_cohesive,HPC 33
1054,23-14-A,cold_discohesive,HPC 30
1055,23-14-D,cold_discohesive,NaN
1056,23-14-J,cold_cohesive,HPC 11


### Subclassify HD

In [18]:
supercluster_pure_cores_HD_split = supercluster_pure_cores.copy(deep=True)
supercluster_pure_cores_HD_split['highrisk_HD'] = supercluster_pure_cores_HD_split.apply(lambda row: row['cluster_13'], axis=1)
supercluster_pure_cores_HD_split['lowrisk_HD'] = supercluster_pure_cores_HD_split.apply(lambda row: row['cluster_10'] + row['cluster_16'] + row['cluster_28'] + row['cluster_61'] + row['cluster_48'] + row['cluster_69'], axis=1)
supercluster_pure_cores_HD_split['consensus_HD_equal'] = supercluster_pure_cores_HD_split.apply(lambda row: 1 if row['highrisk_HD'] >= row['lowrisk_HD'] else 0, axis=1)
supercluster_pure_cores_HD_split['consensus_HD_more'] = supercluster_pure_cores_HD_split.apply(lambda row: 1 if row['highrisk_HD'] > row['lowrisk_HD'] else 0, axis=1)
supercluster_pure_cores_HD_split

,core_ID,cluster_0,cluster_5,cluster_11,cluster_13,cluster_17,cluster_21,cluster_27,cluster_28,cluster_30,...,hot_discohesive,cold_cohesive,cold_discohesive,sc_max,supercluster,cluster,highrisk_HD,lowrisk_HD,consensus_HD_equal,consensus_HD_more
0,01-02-A,0,0,0,0,0,0,0,0,0,...,4,0,0,4,hot_discohesive,NaN,0,4,0,0
1,01-02-B,0,0,0,0,0,0,0,0,0,...,0,1,3,3,cold_discohesive,HPC 64,0,0,1,0
2,01-02-C,0,0,0,0,0,0,1,0,0,...,0,0,4,4,cold_discohesive,NaN,0,0,1,0
3,01-02-D,0,0,0,0,0,0,1,0,0,...,0,1,3,3,cold_discohesive,NaN,0,0,1,0
4,01-02-F,0,1,1,0,0,0,0,0,0,...,0,1,2,2,cold_discohesive,NaN,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,23-13-H,0,0,0,0,0,0,0,0,0,...,0,3,0,3,cold_cohesive,HPC 33,0,0,1,0
1054,23-14-A,0,0,0,0,0,0,0,0,3,...,0,0,3,3,cold_discohesive,HPC 30,0,0,1,0
1055,23-14-D,0,0,0,0,0,0,0,0,0,...,0,0,2,2,cold_discohesive,NaN,0,0,1,0
1056,23-14-J,0,0,3,0,0,0,0,0,0,...,0,3,0,3,cold_cohesive,HPC 11,0,0,1,0


In [19]:
supercluster_pure_cores_HD_split[supercluster_pure_cores_HD_split['supercluster'] == 'hot_discohesive']['consensus_HD_equal'].value_counts()

0    255
1     28
Name: consensus_HD_equal, dtype: int64

In [20]:
supercluster_pure_cores_HD_split[supercluster_pure_cores_HD_split['supercluster'] == 'hot_discohesive']['consensus_HD_more'].value_counts()


0    261
1     22
Name: consensus_HD_more, dtype: int64

In [21]:
supercluster_pure_cores_HD_split[['core_ID', 'supercluster', 'cluster', 'consensus_HD_equal', 'consensus_HD_more']].to_csv('/nfs/home/users/krakovic/sharedscratch/notebooks/latticea_he/final_figures/bioclavis/data/bioclavis_pure_superclusters_split_HPC13.csv')